In [30]:
from rtk.task import KrakenAltoCleanUpCommand, YALTAiCommand, KrakenRecognizerCommand, ExtractZoneAltoCommand
from rtk import utils
import glob
from sys import argv
import time
import os

In [47]:
# Get all PNG files from all subdirectories
all_files = glob.glob("Editions/Padova_1618_Cesare_Ripa_processed/*.png")


print(f"processing {len(all_files)} PNGs")

start = time.time()
if len(argv) == 2:
    num_workers = int(argv[1])
else:
    num_workers = 5

for i in range(0, len(all_files), 4):
    batch = all_files[i:i+4]
    
    startYalt = time.time()
    # Apply YALTAi
    print("[Task] Segment, size of batch ", len(batch))
    yaltai = YALTAiCommand(
        batch,
        binary="yaltai",
        device="cpu",
        yolo_model="models/ladas-1280-l.pt",
        verbose=True,
        raise_on_error=False,
        allow_failure=False,
        multiprocess=10,
        check_content=False,
        line_model="models/blla.mlmodel"
    )
    yaltai.process()
    endYalt = time.time()
    
    print("[Time] Yaltai: ", endYalt - startYalt)
    print("Yaltai output files len ", len(yaltai.output_files))
    print("[Task] Fix ALTO file paths") 
    
    # Fix paths for current batch - corrected for PNG files
    batch_xml_files = [img.replace('.png', '.xml') for img in batch]
    for alto_file in batch_xml_files:
        if os.path.exists(alto_file):
            with open(alto_file, 'r') as f:
                content = f.read()
            
            # Get the corresponding image file name
            img_file = alto_file.replace('.xml', '.png')
            img_filename = os.path.basename(img_file)
            
            # Fix various possible path patterns
            fixed_content = content.replace(f'<fileName>{img_file}</fileName>', f'<fileName>{img_filename}</fileName>')
            fixed_content = fixed_content.replace(f'<fileName>Editions/Padova_1618_Cesare_Ripa_processed/', '<fileName>')
            fixed_content = fixed_content.replace('<fileName>sample/', '<fileName>')
            
            # Remove any remaining directory structure
            import re
            fixed_content = re.sub(r'<fileName>[^<]*?([^/]+\.png)</fileName>', r'<fileName>\1</fileName>', fixed_content)
            
            with open(alto_file, 'w') as f:
                f.write(fixed_content)
    
    print("[Task] Clean-Up Serialization")
    cleanup = KrakenAltoCleanUpCommand(yaltai.output_files)
    cleanup.process()
    
    startKrak = time.time()
    print("[Task] OCR")
    kraken = KrakenRecognizerCommand(
        yaltai.output_files,
        binary="kraken",
        device="cpu",
        model="models/catmus-print-fondue-large.mlmodel",
        multiprocess=10,
        check_content=True
    )
    kraken.process()
    endKrak = time.time()
    
    print("[Time] Kraken: ", endKrak - startKrak)

end = time.time()
print("[Time] total: ", end - start)

Found 704 PNG files to process
[Task] Segment, size of batch  4



[Subtask] Checking prior processed documents: 100%|██████████| 4/4 [00:00<00:00, 23140.99it/s]

[Subtask] Processing YALTAi segmenter command: 100%|██████████| 4/4 [00:20<00:00,  5.03s/it]


[Time] Yaltai:  20.1311616897583
Yaltai output files len  4
[Task] Fix ALTO file paths
[Task] Clean-Up Serialization



[Subtask] Checking prior processed documents: 100%|██████████| 4/4 [00:00<00:00, 1630.60it/s]


Nothing to process here.
[Task] OCR


...

[Time] Yaltai:  20.70979595184326
Yaltai output files len  4
[Task] Fix ALTO file paths
[Task] Clean-Up Serialization



[Subtask] Checking prior processed documents: 100%|██████████| 4/4 [00:00<00:00, 2048.25it/s]


Nothing to process here.
[Task] OCR



[Subtask] Checking prior processed documents: 100%|██████████| 4/4 [00:00<00:00, 2457.12it/s]

[Subtask] Processing Kraken recognizer command: 100%|██████████| 4/4 [00:33<00:00,  8.36s/it]

[Time] Kraken:  33.425166845321655
[Time] total:  8811.147813081741
